In [5]:
from IPython.display import display, Math, Latex

import pandas as pd
import numpy as np
import numpy_financial as npf
import yfinance as yf
import matplotlib.pyplot as plt
from datetime import datetime

#### Group Assignment
- Team Number: <font color="yellow"> 12 </font>
- Team Member Names: <font color="yellow"> Robin Zhang, Jessica Chen, Eric Wang </font>
- Team Strategy Chosen: <font color="red"> Risky </font>

In [11]:
start_date = "2017-01-01"
end_date = "2022-11-02"

In [1]:
class Stock:

    # Initializing values when class is created
    def init(self, stockList, start, end):
        
        self.stockList = stockList
        self.start = start
        self.end = end
        self.dateRange = pd.date_range(start=self.start, end=self.end, freq=pd.offsets.Week(1))
        self.makeDataDict()
        self.makeDataFrame()
        self.makeColumns()


    # Takes the history of the ticker and puts in dictionary
    def makeDataDict(self):
        self.companies = {}
        for ticker in self.stockList:
            self.companies[ticker] = yf.Ticker(ticker).history(start = self.start, end = self.end)['Close']

    # Creates a DataFrame for every day 1 of the month
    def makeDataFrame(self):
        self.companiesDF = pd.DataFrame(self.companies)
        self.companiesDF = self.companiesDF.groupby(pd.DatetimeIndex(self.companiesDF.index).to_period('W')).nth(0)

    def makeColumns(self):
        self.columns = []
        for column in self.companiesDF.iteritems():
            self.columns.append(column[0])

    # Graphs the price of the Stocks
    def graphStocks(self):
        for x in self.columns:
            plt.plot(self.dateRange, self.companiesDF[x], label = x)
        plt.title("Portfolio Stock Prices", fontsize=20)
        self.makeGraph(True)

    # Adds the Axis and Customization to Graphs
    def makeGraph(self, legend=False):
        plt.xticks(rotation=70)
        plt.xlabel("Dates", fontsize=14)
        plt.ylabel("Price $(USD)", fontsize=14)
        if legend:
            plt.legend(bbox_to_anchor=(1.0, 1.0))

In [24]:
# currently not working with class objects
def calculateBeta(tickerList):
    '''
    purpose:
        calculates beta between each individual stock and the overall market (SP500 index)
    input:
        tickerList (listof Stock): a list of Stock objects
    output: 
        A list of the beta of each stock in tickerList (assume US market)
    '''
    beta = dict() # to store all beta values

    # calculating market value
    marketIndex='^GSPC'
    marketTicker=yf.Ticker(marketIndex)
    marketHist=marketTicker.history(start=start_date, end=end_date, interval="1mo")
    marketVar=marketHist["Close"].pct_change().var()
    
    # calculating each Beta, then store in the beta dict
    for ticker in tickerList:
        
        #delete later, just access object attributes (make sure it's PERCENT CHANGE)
        tempTicker=yf.Ticker(ticker)
        tempHist=tempTicker.history(start=start_date, end=end_date, interval="1mo")
        temp=pd.DataFrame()
        temp["ticker_close"]=tempHist["Close"].pct_change()
        temp['market_close']=marketHist["Close"].pct_change()
        temp.dropna(inplace=True)

        tempBeta=temp.cov()/marketVar
        #make sure to access the name attribute later
        tempBeta=tempBeta.iloc[0,1]
        beta[ticker]=tempBeta
    return beta
        
calculateBeta(["AAPL", "SHOP", "NFLX"])

{'AAPL': 1.2581464115427943,
 'SHOP': 1.8598595513180114,
 'NFLX': 1.300868723529694}

In [34]:
def calculateCorrelation(ticker1, ticker2):
    '''
    purpose:
        calculates the correlation between two stock
    input:
        ticker1 (Stock): Stock object 1
        ticker2 (Stock): Stock object 2
    output: 
        A Dictionary with key of format "ticker1+ticker2" and the 
        value is the correlation between the 2 tickers
    '''
    # change to accessing object attribute
    tickerOne=yf.Ticker(ticker1)
    tickerOneHist=tickerOne.history(start=start_date, end=end_date, interval="1mo")
    tickerTwo=yf.Ticker(ticker2)
    tickerTwoHist=tickerTwo.history(start=start_date, end=end_date, interval="1mo")

    correlation=pd.DataFrame()
    correlation[ticker1]=tickerOneHist["Close"].pct_change()
    correlation[ticker2]=tickerTwoHist["Close"].pct_change()
    correlation.dropna(inplace=True)
    correlation=correlation.corr()
    correlation=correlation.iloc[0,1]

    return {ticker1+"+"+ticker2: correlation}

calculateCorrelation("AAPL", "NFLX")

{'AAPL+NFLX': 0.38525161813324565}

In [25]:
def selectPortfolio(beta, tickers):
    print()

## Contribution Declaration

The following team members made a meaningful contribution to this assignment:

- Robin:
- Jessica:
- Eric: 